# Another Dash app, this time for spaceX compapny

###first we download and import necessarry packages

In [68]:
!pip install dash

In [69]:
import dash
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

### load the data set we are working with

In [70]:
df= pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


make the slices of df that will be needed in the plotly graphs

In [71]:
df_sites= df.groupby('Launch Site')['class'].sum().reset_index()

# first not working approach, eventually plotly pie cant handle more than 1 column per entry.
# although is was a valid attempt it just need more complex code that can't be debuged
df_sites1= df[df['class']==0].groupby('Launch Site')['class'].count().reset_index()
df_sites1= pd.DataFrame(df_sites1)
df_sites2= df[df['class']==1].groupby('Launch Site')['class'].count().reset_index()
df_sites2= pd.DataFrame(df_sites2)
df_sites2.rename(columns={'class': 'successful launch'},inplace=True)
df_sites1.rename(columns={'class': 'failed launch'},inplace=True)
df_sites3= pd.merge(df_sites1, df_sites2,on='Launch Site')

df_sites3.drop_duplicates()
df_sites3.head()

,Launch Site,failed launch,successful launch
0,CCAFS LC-40,19,7
1,CCAFS SLC-40,4,3
2,KSC LC-39A,3,10
3,VAFB SLC-4E,6,4


In [113]:
# easier and more valid attempt, here each site has two values and their counts. something like melt function can do i guess
success_fail_count= df.groupby('Launch Site')['class'].value_counts().reset_index()
success_fail_count= pd.DataFrame(success_fail_count)
success_fail_count['class'].replace({0: 'Failed', 1: 'Success'}, inplace=True)
success_fail_count

,Launch Site,class,count
0,CCAFS LC-40,Failed,19
1,CCAFS LC-40,Success,7
2,CCAFS SLC-40,Failed,4
3,CCAFS SLC-40,Success,3
4,KSC LC-39A,Success,10
5,KSC LC-39A,Failed,3
6,VAFB SLC-4E,Failed,6
7,VAFB SLC-4E,Success,4


In [114]:
# slice the previous df to each site
site1= success_fail_count.iloc[0:2, :]
site2= success_fail_count.iloc[2:4, :]
site3= success_fail_count.iloc[4:6, :]
site4= success_fail_count.iloc[6:8, :]


In [131]:
df_scatter0= df.groupby('Launch Site')[['Payload Mass (kg)']].sum().reset_index()
df_scatter1= df.groupby('Launch Site')[['class']].mean().reset_index()

df_scatter= pd.merge(df_scatter0, df_scatter1, on='Launch Site')
df_scatter

,Launch Site,Payload Mass (kg),class
0,CCAFS LC-40,67363.00,0.269231
1,CCAFS SLC-40,24616.65,0.428571
2,KSC LC-39A,56894.65,0.769231
3,VAFB SLC-4E,58138.00,0.400000


### Making the interactive dash app, and have fun!

In [ ]:
app = dash.Dash(__name__)
app.layout = html.Div(children=[html.H1('Space X statistics visuals',
                                style={'textAlign': 'center', 'color': 'lightgreen', 'font_size': 25}),

                        html.Div([
                              html.Label('Potato...'),
                              dcc.Dropdown(id='site_dropdown',
                                          options=[{'label':'All sites', 'value':'ALL'},
                                                  {'label':site, 'value':site} for site in df_sites['Launch Site'].values()],
                                          value='ALL')],

                                                              html.Div([html.H1('Pie chart for all sites....')
                                                                  html.Div(dcc.Graph(id= 'pie_chart'), figure= pie_chart)
                                                              ]))])

@app.callback(Output('pie_chart','figure'),
              Input('site_dropdown','value'))

def update_pie_chart(site):
    if site == 'ALL':
        fig1= px.pie(df_sites, values='class', names='Launch Sites', title= 'Success launches for all sites')
        return fig1
    else:
        fig1= px.pie(df_sites, values='class', names= site for site in df_sites['Launch Site'].values(), title= (f'Success launches for' site))
        return fig1


if __name__ =='__main__':
    app.run_server(debug=True)

SyntaxError: did you forget parentheses around the comprehension target? (<ipython-input-38-f749a1539489>, line 8)

In [137]:

app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1('Space X Statistics Visuals',
            style={'textAlign': 'center', 'color': 'lightgreen', 'fontSize': 25}),

    html.Div([
        html.Label('Select Launch Site:'),
        dcc.Dropdown(id='site_dropdown',
                     options=[{'label': 'All sites', 'value': 'ALL'}] +
                             [{'label': site, 'value': site} for site in df_sites['Launch Site'].unique()],
                     value='ALL')
    ]),

    html.Div([
        html.H1('Pie Chart for Launch Sites'),
        dcc.Graph(id='pie_chart')
    ]),

    html.Div(
             dcc.Graph(id='scatter_plot')
             ),
    html.Div([
        dcc.RangeSlider(id= 'payload_slider',
                        min= 0,max= 10000,step= 1000,
                        value= [df['Payload Mass (kg)'].min(), df['Payload Mass (kg)'].max()])
    ])
])



@app.callback(
    Output('pie_chart', 'figure'),
    Input('site_dropdown', 'value')
)

def update_pie_chart(site):
    if site == 'ALL':
        fig = px.pie(df_sites, values='class', names='Launch Site',
                      title='Success launches for all sites')
        return fig
    else:
        if site == 'CCAFS LC-40':
            fig1= px.pie(site1, names= 'class', values= 'count',title= 'Success-failure rate of site: CCAFS LC-40')
            return fig1
        elif site == 'CCAFS SLC-40':
            fig2= px.pie(site2, names= 'class', values= 'count', title= 'Success-failure rate of site: CCAFS SLC-40')
            return fig2
        elif site == 'KSC LC-39A':
            fig3= px.pie(site3, names= 'class', values= 'count', title= 'Success-failure rate of site: KSC LC-39A')
            return fig3
        else:
            fig4= px.pie(site4, names= 'class', values= 'count', title= 'Success-failute rate of site: VAFB SLC-4E')
            return fig4

@app.callback(
    Output('scatter_plot', 'figure'),
    Input('payload_slider', 'value'),
    Input('site_dropdown', 'value')
)

def update_scatter_plot(payload,site):
    low, high= payload
    if site == 'ALL':
        filtered_df = df[(df['Payload Mass (kg)'] >= low) & (df['Payload Mass (kg)'] <= high)]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title='Success launches for all sites')
        return fig
    else:
        filtered_df = df[df['Launch Site']==site]
        filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= low) & (filtered_df['Payload Mass (kg)'] <= high)]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title='Success launches for all sites')
        return fig



if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

In [133]:
fig = px.scatter(df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title='Success launches for all sites')
fig.show()

In [88]:
fig= px.pie(df_site1, names= 'class', values= 'count', title= 'test potato')
fig.show()